In [10]:
# initiating some packages
import cv2
import numpy as np
import pandas as pd
from skimage.filters import threshold_otsu
import os
import bigfish
import bigfish.stack as stack
import bigfish.detection as detection
import bigfish.multistack as multistack
import bigfish.plot as plot
import matplotlib.pyplot as plt
import tifffile as tiff
import datetime
from scipy import ndimage
from skimage.morphology import skeletonize
def current_time():
#get the current date and time
    now=datetime.datetime.now()
#format date as a string
    date_str=now.strftime('%Y-%m-%d')
    return date_str
# Set the path to the main directory containing the subdirectories with the images
path = r'F:\TRPV1_12072023'
current_time=current_time()
results_path = r"F:\TRPV1_12072023\trpv1_sub25.csv"
if not os.path.exists(results_path):
    
# Create an empty DataFrame with the desired column names
#column = ['Name', 'Amount', 'pgp_area', 'pgp_int', 'trpv1_int', 'total_dots', 'trpv1_area', 'trpv1_percent_pgp', 'Threshold']
    column = [  'Name', 
                'Amount', 
                'pgp_area', # default unit is pixel, 
                'total_dots',
                'Threshold',
               
             ]
    results_df = pd.DataFrame(columns=column)
# Save the empty DataFrame to the results file with headers
    results_df.to_csv(results_path, index=False)
# mean fluorescence intensity 
def get_intensity(image,mask):
    masking=mask>0
    # same with RawIntD in ImageJ, masked image is RGB image then to sum up will get integrated intensity
    masked=image*masking 
    area_micro = np.sum(masking)* (pixel_convert_micro *pixel_convert_micro)
    #np.sum(masked) : the integrated intensity .
    #np.sum(masked)/np.sum(masking) :the mean fluorescence intensity .
    return np.sum(masked)/np.sum(masking),np.sum(masked),area_micro # np.sum(masked) stand for integrated intensity
def count_dots_and_area(trpv1_image,mask_image,threshold):
    # define the structure for dilation
    structure = np.ones((0,0),np.uint8)
    # perform binary dilation
    dilated_mask = cv2.dilate(mask_image, structure, iterations = 1)
    # transfer the mask into uint8 type
    dilated_mask = dilated_mask.astype('uint8')
    masked_binary_image=trpv1_image*dilated_mask
    # set the parameter including threshold,log_kernel_size,and minimum_distance
    spots, threshold = detection.detect_spots(
    images=masked_binary_image, 
    return_threshold=True, 
    threshold=threshold,
    log_kernel_size=1.0,
    minimum_distance=9)
    n_dots=len(spots)
    
    return n_dots, masked_binary_image
# Loop through each subdirectory in the main directory
for dirpath, dirnames, filenames in os.walk(path):  
    # Check if the current subdirectory contains the PGP, TRPV1, and mask subdirectories
    if 'PGP' in dirnames and 'TRPV1' in dirnames and 'PGP_pred' in dirnames:
        sample_name = os.path.basename(dirpath)
        pixel_convert_micro = 0.16
        # please set the threshold you want,default threshold is 30.
        for threshold in range(200, 201):   
            # Get the corresponding filenames from the PGP, TRPV1, and mask subdirectories
            pgp_filenames = [f for f in os.listdir(os.path.join(dirpath, 'PGP')) if f.endswith('.tif')]
            trpv1_filenames = [f for f in os.listdir(os.path.join(dirpath, 'TRPV1')) if f.endswith('.tif')]
            mask_filenames = [f for f in os.listdir(os.path.join(dirpath,"PGP_pred" ,'masks')) if f.endswith('.png')]
            epi_filenames = [f for f in os.listdir(os.path.join(dirpath, 'DAPI_epi', 'masks')) if f.endswith('.png')]
            sub_filenames = [f for f in os.listdir(os.path.join(dirpath, 'DAPI_sub', 'masks')) if f.endswith('.png')]
            total_pgp_area=0
            total_dots=0
            total_pgp_area=0
            total_sub_area=0
            time=0
            # Loop through each corresponding filename and read the images
            for pgp_filename, trpv1_filename, mask_filename, epi_filename, sub_filename in zip(pgp_filenames, trpv1_filenames, mask_filenames, epi_filenames, sub_filenames):
                # pgp,trpv1,mask path
                pgp_path = os.path.join(dirpath, 'PGP', pgp_filename)    
                trpv1_path = os.path.join(dirpath, 'TRPV1', trpv1_filename)
                mask_path = os.path.join(dirpath, "PGP_pred" ,'masks', mask_filename)
                sub_path = os.path.join(dirpath, 'DAPI_sub', 'masks', sub_filename)
                epi_path = os.path.join(dirpath, 'DAPI_epi', 'masks', epi_filename)
                # sample number
                filename = os.path.basename(pgp_path)
                pre_prefix = os.path.splitext(filename)[0]
                prefix=pre_prefix[-7:]
                # read pgp,trpv1,mask images. pgp and trpv1 default form are 'uint16', masks are 'float32'
                pgp_image= plt.imread(pgp_path)
                trpv1_image= plt.imread(trpv1_path)
                mask_image=cv2.imread(mask_path)             
                epi_mask = cv2.imread(epi_path)              
                sub_mask = cv2.imread(sub_path)
                #convert 'float32' into 'uint8'
                mask_image = mask_image[:,:,0].astype('uint8')
                epi_mask = epi_mask[:,:,0].astype('uint8')
                sub_mask = sub_mask[:,:,0].astype('uint8')            
                # dilate epidermi's mask,define the depth of dermis
                dilated_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (306, 306))
                dilated_mask = cv2.dilate(epi_mask, dilated_kernel, iterations=1)
                mask1_image = cv2.bitwise_and(dilated_mask, sub_mask)  
                mask1_image = cv2.bitwise_xor(mask1_image, epi_mask)  
                mask1_image = cv2.bitwise_and(mask1_image, sub_mask)  
                # Save the image using cv2.imwrite
                cv2.imwrite(f'{dirpath}\ROI50_mask_{prefix}.png', mask1_image)        
                nerve_sub_mask =cv2.bitwise_and(mask1_image, mask_image) 
                # Threshold the color image to create a binary mask
                _, binary_mask = cv2.threshold(nerve_sub_mask, 0, 1, cv2.THRESH_BINARY)

                 # function def get_intensity to measure the integrated intensity 
                integrate_pgp_int= get_intensity(pgp_image,nerve_sub_mask) [1]# mean intensity of PGP on PGP-mask
                integrate_trpv1_int= get_intensity(trpv1_image,nerve_sub_mask)[1]  # mean intensity of TRPV1 on PGP-mask
                
                # measure the total size area of mask
                #pixel_convert_micro = 0.16 # microns depend on what the image scale is 
                pgp_area=get_intensity(pgp_image,nerve_sub_mask)[2]
                sub_area=get_intensity(pgp_image,mask1_image)[2]
                total_pgp_area= total_pgp_area + pgp_area
                total_sub_area=total_sub_area + sub_area
                
                time= time+ 1

                # measurement the amount of TRPV1 dots

                dots = count_dots_and_area(trpv1_image, nerve_sub_mask,threshold)[0]
                print(dots)
                total_dots= total_dots + dots
                masked_binary_image =count_dots_and_area(trpv1_image, nerve_sub_mask,threshold)[1]
             
            # Print and save the results, keep decimal 2  
            mean_pgp_area= f'{(total_pgp_area/ time):.2f}'
            mean_trpv1_area=f'{(total_trpv1_area/ time):.2f}'
            
            mean_pgp_int=(total_integrate_pgp_int * pixel_convert_micro**2)  / total_pgp_area  #sum of intensity per micrometer^2
            mean_trpv1_int= (total_integrate_trpv1_int * pixel_convert_micro**2)/ total_pgp_area
            
            
            # read the existing csv.file
            results_df=pd.read_csv(results_path)
            # create new data to existing file
            results_dict = { 'Name':[sample_name], 
                            'Amount': [time], 
                            'pgp_area':[mean_pgp_area],
                            'total_dots':[total_dots/time],
                            'Threshold':[threshold],
                           
                            }
            results_df = pd.DataFrame.from_dict(results_dict)
            results_df.to_csv( results_path, mode='a', index=False, header=False)
            
            print(results_df)

25
26
36
46
32
     Name  Amount pgp_area  total_dots  Threshold
0  100020       5    91.90        33.0        200
51
32
62
51
70
108
     Name  Amount pgp_area  total_dots  Threshold
0  100221       6   184.44   62.333333        200
5
25
11
24
     Name  Amount pgp_area  total_dots  Threshold
0  101020       4    39.46       16.25        200
13
7
20
7


KeyboardInterrupt: 